In [1]:
import taichi as ti
ti.init(arch=ti.cuda)

[Taichi] version 1.0.3, llvm 10.0.0, commit fae94a21, win, python 3.9.12
[Taichi] Starting on arch=cuda


In [2]:
# import numpy as np

def chain_array(A):
    B = A[:-1]
    E = A[1:]
    return B,E

# @ti.kernel
# def advance():
#     for i in x:
#         # accelerate in the direction that decreases potential energy (u)
#         v[i] += dt *-x.grad[i]  # dv/dt = -dU/dx
#     for i in x:
#         x[i] += dt * v[i]  # dx/dt = v

# ropearray = np.random.random((5,2))




# # print(Y)


# gui = ti.GUI("lines", res=(400, 400))
# while gui.running:
    
#     B,E = chain_array(ropearray)
#     gui.lines(begin=B, end=E, radius=2, color=0x068587)
#     gui.show()

In [3]:

N = 10
segment_length = 0.01
dt = 1e-5

x = ti.Vector.field(2, dtype=ti.f32, shape=N)  # particle positions
oldx = ti.Vector.field(2, dtype=ti.f32, shape=N)  
attractor_strength = ti.field(dtype=ti.f32, shape=()) 
gravity = 10

indices = ti.field(dtype=ti.i32,shape=2*N - 1)
count = 0
indices[count]=0
count+=1
for i in range(1,N):
#         count =
    indices[count]=i
    count+=1
    indices[count]=i
    count+=1



@ti.kernel
def advance():

    
    for i in range(0,N):
#         rope constraint (https://youtu.be/FcnvwtyxLds)
#         also looked at the game code for Double Wires which was the original and uses the same physics as spiderman
#         it had unobfuscated code when it was decompiled unlike spiderman

        r = (oldx[i+1]-x[i])
        d = r.norm(1e-6)
        delta_d = segment_length-d
        r *= (delta_d*0.5)/d 
        
        x[i] -= r
#         normally it would just be:
#         x[i+1] += r
#         but doing it this way stops the last node in the rope from applying its force to another node unnecessarily.
#         It also allows for the vector
        x[max((i+1)%N,i)] += r
           
        
#         normal verlet integration
        vector = (x[i]-oldx[i]) + dt * ti.Vector([0,-gravity])
#         print(vector)
        oldx[i] = x[i]
        x[i] += vector
        
        



def substep():
#     with ti.Tape(loss=U):
#         # Kernel invocations in this scope will later contribute to partial derivatives of
#         # U with respect to input variables such as x.
#         compute_U(
#         )  # The tape will automatically compute dU/dx and save the results in x.grad
    advance()


@ti.kernel
def init():
#     x[0]
    for i in range(0,N):
        a = 0.5
        b = 0.5 + 0.01 * -i
        x[i] = [a,b]
        oldx[i]=[a,b]
    x[N] = [0.5,0.5]
    

    


init()
# gui = ti.GUI('Autodiff gravity', (1000, 1000))


res = (512, 512)
window = ti.ui.Window("Taichi MLS-MPM-128", res=res, vsync=True)
canvas = window.get_canvas()


mouse_x = 0.5
mouse_y = 0.6

# count = 0

# canvas = window.get_canvas()
import time
print(indices)
print(x)
start = time.time()
while window.running:

    if window.is_pressed(ti.ui.LMB):
        mouse_x,mouse_y = window.get_cursor_pos()

    advance()
    
    x[N-1] = [mouse_x,mouse_y]

#     mouse_x +=0.01
#     mouse_x = mouse_x % 2
#     x[N-1] = [abs(mouse_x-1),mouse_y]
    
    
    oldx[N-1] = x[N-1]
    
    canvas.set_background_color((0, 0, 0))
    canvas.lines(x, indices=indices, width=0.01,color=(0.2, 0.4, 0.6))
    canvas.circles(x, radius=0.004,color=(1, 1, 1))
    window.show()
    time.sleep(0.0166)



[0 1 1 2 2 3 3 4 4 5 5 6 6 7 7 8 8 9 9]
[[0.5        0.5       ]
 [0.5        0.49      ]
 [0.5        0.48      ]
 [0.5        0.47      ]
 [0.5        0.46      ]
 [0.5        0.45      ]
 [0.5        0.44      ]
 [0.5        0.43      ]
 [0.5        0.42000002]
 [0.5        0.41      ]]


In [4]:
#     for i in range(round(1e-5/dt)):
# #         substep()
#         advance()